# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [31]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [32]:
# Read the .csv file, create a DataFrame and display the first 5 rows
data_df = pd.read_csv("cities.csv")
data_df.head()

City  Cloudiness Country        Date  Humidity    Lat     Lng  \
0       uspenka          83      UA  1585616628        67  48.39   39.16   
1        hobart          40      AU  1585616628        48 -42.88  147.33   
2  norman wells          20      CA  1585616498        45  65.28 -126.83   
3        albany          90      US  1585616629        75  42.60  -73.97   
4    georgetown          20      MY  1585616464        88   5.41  100.34   

   Max Temp  Wind Speed  
0     10.22        3.85  
1     17.00        4.60  
2    -15.00        3.10  
3      7.00        0.67  
4     28.00        2.46

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [33]:
# Data configuration
locations = data_df[["Lat", "Lng"]]
humidity = data_df["Humidity"].astype(float)

# Get the maximum Humidity value to set as the "hottest" zone in the heatmap
max_int = data_df["Humidity"].astype(float).max()

In [34]:
# Plot Humidity heatmap
fig = gmaps.figure()

# Create a heat layer
heat_l = gmaps.heatmap_layer(locations, weights = humidity, max_intensity = max_int, point_radius = 8)

# Add the heat layer
fig.add_layer(heat_l)

# Display the plot
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [35]:
# Reduce the DataFrame to ideal conditions, humidity between 20% and 50%, maximum temperature between 20 (exclusive) and 30
fav_df = data_df[data_df["Humidity"] <= 50][["City", "Country", "Lat", "Lng", "Humidity", "Max Temp"]]
fav_df = fav_df[fav_df["Humidity"] >= 20][["City", "Country", "Lat", "Lng", "Humidity", "Max Temp"]]
fav_df = fav_df[fav_df["Max Temp"] <= 30][["City", "Country", "Lat", "Lng", "Humidity", "Max Temp"]]
hotel_df = fav_df[fav_df["Max Temp"] > 20][["City", "Country", "Lat", "Lng", "Humidity", "Max Temp"]]
hotel_df = hotel_df.dropna()
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 7 to 547
Data columns (total 6 columns):
City        25 non-null object
Country     25 non-null object
Lat         25 non-null float64
Lng         25 non-null float64
Humidity    25 non-null int64
Max Temp    25 non-null float64
dtypes: float64(3), int64(1), object(2)
memory usage: 1.4+ KB


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [36]:
# New column for hotel names
hotel_df["Hotel Name"] = ""

# Search parameters for the API calls
params = {"radius":5000,
          "types":"lodging",
          "key":g_key}

# Url for the google maps API
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Use the Lat and Lng to retrieve the hotel information
for index, row in hotel_df.iterrows():
    
    # Get the current Lat and Lng
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Set the current location
    params["location"] = f'{lat}, {lng}'
    
    # API call (request) using the url and the parameters
    hotel_data = requests.get(url, params=params).json()
    
    # Try code to prevent crashes
    try:
        
        # Set variable to retrieve the name of the first hotel
        name = hotel_data["results"][0]["name"]
        
        # Add the hotel name to its corresponding row
        hotel_df.loc[index, "Hotel Name"] = name
        
        # Print a log confirming the hotel was found in that city
        print(f'Hotel found in {row["City"]}')
    
    # Add exceptions if the hotel can't be found (to prevent crashes)
    except:
              
        # Print a log to let the user know no hotel was found in that city
        print(f'No hotel found in {row["City"]}')

# Código de prueba para narrow down a solo hoteles              
#     for i in range(len(hotel_data["results"])):
#         name = hotel_data["results"][i]["name"]
#         hotel_l = name.lower()
#         if "hotel" in hotel_l != 0:
#             #print(name)
#             hotel_df.loc[index, "Hotel Name"] = name
#             break

# Print a cleaner response
# print(json.dumps(hotel_data, indent=4, sort_keys=True))

Hotel found in samdari
Hotel found in guerrero negro
Hotel found in khategaon
Hotel found in ratanpur
Hotel found in cabo san lucas
No hotel found in avera
No hotel found in nokaneng
Hotel found in umm lajj
Hotel found in garwa
Hotel found in bo phloi
No hotel found in nioro
Hotel found in kapoeta
Hotel found in tignere
Hotel found in aiken
Hotel found in hurghada
Hotel found in aswan
No hotel found in marzuq
Hotel found in havelock
Hotel found in mecca
Hotel found in bhikangaon
Hotel found in columbia
Hotel found in leon
Hotel found in parkes
Hotel found in suleja
Hotel found in saraipali


In [37]:
# Drop the DF row without Hotel and display the first 5 rows
drop_rows = hotel_df[ hotel_df['Hotel Name'] == "" ].index
hotel_df = hotel_df.drop(drop_rows, axis = 0)
hotel_df.head()

City Country    Lat     Lng  Humidity  Max Temp  \
7           samdari      IN  25.82   72.58        27     24.21   
41   guerrero negro      MX  27.98 -114.06        32     23.27   
132       khategaon      IN  22.60   76.92        42     25.60   
134        ratanpur      IN  22.30   82.17        27     23.06   
138  cabo san lucas      MX  22.89 -109.91        38     28.00   

                         Hotel Name  
7                  Karnoba Ki Dhani  
41     Casa Laguna, Bed & Breakfast  
132                  savariya Lodge  
134  Mahamaya lodge & Mangal Bhawan  
138          Hotel Tesoro Los Cabos

In [38]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [39]:
# Add marker layer ontop of heat map
# Stroke is the outer part of the symbol and fill is the inner part
hotel_layer = gmaps.symbol_layer(locations, fill_color = "white", fill_opacity = 0.8, stroke_color = "white", stroke_opacity = 0.1, info_box_content = hotel_info)

# Add the layer to the previous heatmap
fig.add_layer(hotel_layer)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))